# Measuring the Success of Diffusion Models at Imitating Human Artists

TL;DR: We compiled a set of professional digital artists, each with a large amount of copyrighted work online. We found that when Stable Diffusion is prompted to imitate the artists, the artists were classified from the image successfully an average of 82% of the time. Over 10 trials of the experiment, we found that all 70 of the 70 artists were classified from images imitating them at least once.

Stephen Casper,* MIT (scasper@mit.edu)

Zifan Guo,* MIT

Shreya Mogulothu, MIT

Zachary Marinov, MIT

Chinmay Deshpande, Harvard University

Rui-Jie Yew, MIT

Zheng Dai, MIT

Dylan Hadfield-Menell, MIT

## Setting up the experiment environment

In [1]:
%%capture

# install libraries
!pip install -q transformers
!pip install -q diffusers==0.16.1
!pip install -q accelerate

# import libraries
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
import torch
import accelerate
from google.colab import files
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm
from PIL import Image

In [2]:
%%capture

# set up stable diffusion
# download diffusion
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# gets rid of content warnings for more data; can remove
pipe.safety_checker = lambda images, clip_input="": (images, False)
pipe = pipe.to("cuda")

# download clip
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

## Choosing artists for the experiment

In this section, we provide three datasets of artists, one with 70 artists selected through a set of criteria detailed in the writeup, one with 250 artists that have the highest amount of images in the LAION-aesthetics dataset, and one with 70 random names generated for control.

We set each prompt to be "Artwork from [artist's name]". We also use 3 default prompts: ["Artwork", "Digital Artwork", "Artwork from the public domain"] to help ensure that when an artist is selected as a label, it is done so over generic alternative labels.


In [3]:
# 70 artists
artists = ['Adrian Smith', 'Agnes Cecile', 'Alex Andreev', 'Alex Grey', 'Alex Ross',
           'Alexander Jansson', 'Amanda Clark', 'Andreas Rocha', 'Andy Kehoe', 'Anna Dittmann',
           'Anna Razumovskaya', 'Anne Stokes', 'Artgerm', 'Atey Ghailan', 'Audrey Kawasaki',
           'Beeple', 'Brooke Shaden', 'Bruce Pennington', 'Carne Griffiths', 'Casey Weldon',
           'Catrin Welz-Stein', 'Charlie Bowater', 'Cyril Rolando', 'Donato Giancola',
           'Dorina Costras', 'Duy Huynh', 'Filip Hodas', 'Hsiao-Ron Cheng', 'Ian McQue',
           'Ilya Kuvshinov', 'Ismail Inceoglu', 'James Gilleard', 'James Jean', 'Janek Sedlar',
           'Jeremy Geddes', 'Jeremy Mann', 'Jessica Rossier', 'Jimmy Lawlor', 'John Howe',
           'Josephine Wall', 'Kevin Sloan', 'Marc Simonetti', 'Marco Mazzoni', 'Mark Keathley',
           'Mark Ryden', 'Michael Cheval', 'Michael Parkes', 'Michael Whelan', 'Michal Karcz',
           'Miho Hirano', 'Mikko Lagerstedt', 'Naoto Hattori', 'Paul Corfield', 'Peter Gric',
           'Peter Mohrbacher', 'Phil Koch', 'Raphael Lacoste', 'Ray Caesar', 'Scott Naismith',
           'Shaun Tan', 'Simon Stalenhag', 'Sparth', 'Stephan Martiniere', 'Steve Henderson',
           'Ted Nasmith', 'Tom Bagshaw', 'Victo Ngai', 'Victor Nizovtsev', 'Vladimir Kush',
           'Yuumei']


# 250 artists version
# artists = ['Thomas Kinkade', 'Vincent Van Gogh', 'Leonid Afremov', 'Claude Monet', 'Edward Hopper', 'Norman Rockwell', 'William-Adolphe Bouguereau', 'Albert Bierstadt', 'John Singer Sargent', 'Pierre-Auguste Renoir', 'Frida Kahlo', 'John William Waterhouse', 'Winslow Homer', 'Walt Disney', 'Phil Koch', 'Thomas Moran', 'Paul C√©zanne', 'Camille Pissarro', 'Erin Hanson', 'Thomas Cole', 'Raphael', 'Steve Henderson', 'Pablo Picasso', 'Caspar David Friedrich', 'Ansel Adams', 'Diego Rivera', 'Steve McCurry', 'Bob Ross', 'John Atkinson Grimshaw', 'Rob Gonsalves', 'Paul Gauguin', 'James Tissot', 'Edouard Manet', 'Alphonse Mucha', 'Alfred Sisley', 'Fabian Perez', 'Gustave Courbet', 'Zaha Hadid', 'Jean-L√©on G√©r√¥me', 'Carl Larsson', 'Mary Cassatt', 'Sandro Botticelli', 'Daniel Ridgway Knight', 'Joaqu√≠n Sorolla', 'Andy Warhol', 'Kehinde Wiley', 'Alfred Eisenstaedt', 'Gustav Klimt', 'Dante Gabriel Rossetti', 'Tom Thomson', 'Edgar Degas', 'Utagawa Hiroshige', 'Camille Corot', 'Edward Steichen', 'David Hockney', 'Ivan Aivazovsky', 'Josephine Wall', 'Peter Paul Rubens', 'Henri Rousseau', 'Edward Burne-Jones', 'Pixar', 'Alexander McQueen', 'Anders Zorn', 'Jean Auguste Dominique Ingres', 'Franz Xaver Winterhalter', 'Katsushika Hokusai', 'John Constable', 'Canaletto', 'Gordon Parks', 'Shepard Fairey', 'George Inness', 'Anthony van Dyck', 'Vivian Maier', 'Catrin Welz-Stein', 'Lawren Harris', 'Salvador Dali', 'David Bowie', 'Agnes Cecile', 'Titian', 'Martin Johnson Heade', 'Scott Naismith', 'William Morris', 'Berthe Morisot', 'Vladimir Kush', 'William Holman Hunt', 'Edvard Munch', 'Joseph Mallord William Turner', 'Gustave Dor√©', 'Thomas Eakins', 'Ilya Repin', 'Amedeo Modigliani', 'Johannes Vermeer', 'Eyvind Earle', 'Ivan Shishkin', 'Rembrandt Van Rijn', 'Gil Elvgren', 'Nicholas Roerich', 'Henri Matisse', 'Thomas Gainsborough', 'Artgerm', 'Grant Wood', 'Studio Ghibli', 'Jeremy Mann', 'Mark Keathley', 'Maxfield Parrish', 'Andrew Wyeth', 'RHADS', 'David Lynch', 'Frederic Remington', 'Jan Van Eyck', 'Mikko Lagerstedt', 'Banksy', 'Michael Cheval', 'Anna Razumovskaya', 'Jean-Fran√ßois Millet', 'Thomas W Schaller', 'Charlie Bowater', 'El Greco', 'Paolo Roversi', 'Carne Griffiths', 'Man Ray', 'Andrew Macara', 'August Sander', 'Evelyn De Morgan', 'Sally Mann', 'William Blake', 'Oleg Oprisco', 'Yuumei', 'Helmut Newton', 'Henry Ossawa Tanner', 'Asher Brown Durand', 'teamLab', 'August Macke', 'Armand Guillaumin', 'Terry Redlin', 'Antoine Blanchard', 'Anna Ancher', 'Ohara Koson', 'Walter Langley', 'Yayoi Kusama', 'Stan Lee', 'Chuck Close', 'Albert Edelfelt', 'Mark Seliger', 'Eugene Delacroix', 'John Lavery', 'Theo van Rysselberghe', 'Marc Chagall', 'Rolf Armstrong', 'Brent Heighton', 'A.J.Casson', 'Egon Schiele', 'Maximilien Luce', 'Georges Seurat', 'Arthur Hughes', 'George Frederic Watts', 'Anton Mauve', 'Lucian Freud', 'Jessie Willcox Smith', 'Leonardo Da Vinci', 'Edward John Poynter', 'Brooke Shaden', 'J.M.W. Turner', 'Wassily Kandinsky', 'Wes Anderson', 'Jean-Honor√© Fragonard', 'Amanda Clark', 'Tom Roberts', 'Antonello da Messina', 'Hayao Miyazaki', 'Makoto Shinkai', 'Alfred Stevens', 'Slim Aarons', 'Albert Lynch', 'Andre Kohn', 'Daniel Garber', 'Jacek Yerka', 'Beatrix Potter', 'Rene Magritte', "Georgia O'Keeffe", 'Isaac Levitan', 'Frank Lloyd Wright', 'Gustave Moreau', 'Ai Weiwei', 'Ford Madox Brown', 'Tim Burton', 'Alfred Cheney Johnston', 'Duy Huynh', 'Michael Parkes', 'Tintoretto', 'Archibald Thorburn', 'Audrey Kawasaki', 'George Lucas', 'Arthur Streeton', 'Albrecht Durer', 'Andrea Kowch', 'Dorina Costras', 'Alex Ross', 'Hasui Kawase', 'Lucas Cranach the Elder', 'Antonio Mora', 'Briton Rivi√®re', 'Mandy Disher', 'Henri-Edmond Cross', 'Auguste Toulmouche', 'Hubert Robert', 'Syd Mead', 'Alyssa Monks', 'Carl Spitzweg', 'Edward Lear', 'Ralph McQuarrie', 'Sailor Moon', 'Simon Stalenhag', 'Edward Robert Hughes', 'Jules Bastien-Lepage', 'Richard S. Johnson', 'Rockwell Kent', 'Sparth', 'Arnold Bocklin', 'Arnold B√∂cklin', 'Lovis Corinth', 'Robert Hagan', 'Gregory Crewdson', 'Abbott Handerson Thayer', 'Thomas Benjamin Kennington', 'Gilbert Stuart', 'Louis Comfort Tiffany', 'Raphael Lacoste', 'Janek Sedlar', 'Jean Marc Nattier', 'Sherree Valentine Daines', 'Alexander Jansson', 'James Turrell', 'Alex Grey', 'Henri De Toulouse Lautrec', 'Anton Pieck', 'Andrew Atroshenko', 'Ramon Casas', 'Andy Kehoe', 'Andreas Achenbach', 'Eric Zener', 'H.P. Lovecraft', 'Kunisada', 'Jimmy Lawlor', 'Quentin Tarantino', 'Marianne North', 'Vivienne Westwood', 'Tom Bagshaw', 'Jeremy Lipking', 'John Martin']

# 70 random names for control from https://randomwordgenerator.com/name.php
# artists = ['Ariel Case', 'Wilfredo Flynn', 'Madeleine Tapia', 'Dion Mason', 'Aimee Donovan', 'Aiden Rhodes',
#               'Gena Levy', 'Eleanor Monroe', 'Faye Mccarty', 'Jospeh Trevino', 'Leandro Hoover', 'Cary Haney',
#               'Hannah Carney', 'Claire Blackburn', 'Rodrick Carter', 'Alphonse Kelle', 'Zane Ray', 'Kristi Gregory',
#               'Leonor Garrett', 'Sergio Randall', 'Garfield Ingram', 'Vanessa Christian', 'Morton Burch',
#               'Abby Stokes', 'Marion Delacruz', 'Guillermo Chavez', 'Sid Quinn', 'Maynard Wilson', 'Jan Perez',
#               'Mose Weber', 'Tessa Sanchez', 'Dorsey Robbins', 'Maryellen Davies', 'Dario Hardy', 'Maxine Tucker',
#               'Amalia Velazquez', 'Ron Padilla', 'Margo Pace', 'Jorge Scott', 'Brett Pham', 'Bonita Good', 'Sara York',
#               'Randolph Stevens', 'Jefferson Rubio', 'Clair Glenn', 'Sang Jimenez', 'Blanche Zimmerman',
#               'Kristina Hampton', 'Jamal Cherry', 'Fanny Hunter', 'Walton Schultz', 'Hung Martinez',
#               'Marcus Phillips', 'Rose Nelson', 'Jerri Jacobs', 'Bertie Turner', 'Hubert Allen',
#               'Corine Solis', 'Pauline Hobbs', 'Tabitha Wright', 'Catherine Joseph', 'Eva Bullock',
#               'Chuck Branch','Seth Stevens','Gilberto Atkins','Desmond Watson','Ahmad Mcmahon','Vince Gillespie',
#               'Rupert Schroeder','Ines Woods']

artists = sorted(artists)

# prompt for each artist (can be customized)
prompts = [f"Artwork from {artist}" for artist in artists]
default_labels = ["Artwork", "Digital Artwork", "Artwork from the public domain"]
all_labels = prompts + default_labels

## Running the experiment

In [4]:
# run the experiment N times
N = 10

# parameters (most people wont need to worry about thest)
images_per_pipeline = 7  # this helps with RAM issues
cumulative_count = 0
artists_corectly_guessed = set()
# expected artist should be equal to index
expected = torch.arange(len(prompts))
result = []
example_images = []
all_best_guesses = []

with torch.no_grad():

    for i in tqdm(range(N)):
        # need to break it up bc even with "accelerate" you still run into memory issues
        images = []
        for index in range(int(len(prompts)/images_per_pipeline)+2):
            prompt_portion = prompts[images_per_pipeline*index:images_per_pipeline*(index+1)]
            if prompt_portion:
                images += pipe(prompt_portion).images

        # save images for plotting at final iteration
        if i == N-1:
            example_images = images

        # now let CLIP caption them using the prompts inserted. combine images1-4
        # also change prompts for CLIP captioning
        inputs = processor(text=all_labels, images=images, return_tensors="pt", padding=True) # pt means "pytorch"

        # the results
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
        probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
        # best guess has highest probability
        best_guess = torch.argmax(probs, dim=1)
        all_best_guesses.append(best_guess)
        correct_guess_prob = probs.gather(1, expected.unsqueeze(0).t())
        mean_guess_confidence = correct_guess_prob.squeeze(1).mean(dtype=torch.float)

        bgl = best_guess.tolist()
        correct_guess = [bgl[i] for i in range(len(prompts)) if bgl[i]==i]
        artists_corectly_guessed.update(set(correct_guess))
        #   print(f"best guess: {best_guess}")
        #   print(f"correct guess mean confidence: {mean_guess_confidence}")
        best_diff = best_guess - expected
        best_count = len(prompts) - torch.count_nonzero(best_diff)
        cumulative_count += best_count
        top3_guesses = torch.topk(probs, 3, dim=1).indices
        top3_diff = torch.sub(top3_guesses, expected[:, None])
        top3_count = torch.count_nonzero((top3_diff == 0).sum(dim=1))

        #   print(f"best guess mean: {best_count / len(prompts)}")
        #   print(f"top 3 guess mean: {top3_count / len(prompts)}")
        #   print(f"cumulative average: {cumulative_count / (len(prompts) * (i + 1))}")
        result.append((best_count, top3_count, mean_guess_confidence, cumulative_count))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 10/10 [1:20:34<00:00, 483.49s/it]


In [7]:
overall_best_guesses = torch.mode(torch.stack(all_best_guesses), dim=0)[0]
overall_best_guess_count = sum([i == overall_best_guesses[i] for i in range(len(overall_best_guesses))]).sum()

print(f"Number of artists classified correctly at least once {len(artists_corectly_guessed)} out of total {len(prompts)}")
print(f"Number of artists classified correctly a plurality of the time {overall_best_guess_count} out of total {len(prompts)}")
print(f'Overall mean top-1 accuracy: {result[-1][-1] / (len(prompts) * N)}')
print(f'Overall mean confidence in the correct artist: {sum([r[-2] for r in result]) / N}')

Number of artists classified correctly at least once 69 out of total 70
Number of artists classified correctly a plurality of the time 69 out of total 70
Overall mean top-1 accuracy: 0.8100000023841858
Overall mean confidence in the correct artist: 0.6877254247665405


In [6]:
n_col = 10
# setting the size; can be adjusted
plt.figure(figsize=(30, int(len(images) / n_col) * 5))
# plot each image in correct location with title given by artist name
for j in range(len(images)):
    plt.subplot(int(len(images) / n_col) + 1, n_col, j + 1)
    plt.imshow(images[j].resize((100, 100)))
    if j < len(images):
        plt.title(artists[j], fontsize=13, fontweight='bold')
        plt.xticks([])
        plt.yticks([])
plt.show()

Output hidden; open in https://colab.research.google.com to view.